### NOMIS API Data Extract for Historic Census Data.

The NOMIS RESTful API limits the size of the 'queries / calls' you can request.
For example, you cannot request all of the geography codes in a single call, the site will time-out or error due to the number of data items. Additionally each call is limited to a maximum of 25k records; hence the need for a few nested programming loops etc. etc. etc.

***Here are is a useful reading resource:***
https://www.nomisweb.co.uk/api/v01/help

***Here is the target data Darren Barnes has requested:***

"*Had a chat with census team this morning and this is the dataset they would like us to test out
Its a 2011 census table LC3409EW >> https://www.nomisweb.co.uk/census/2011/lc3409ew and includes General Health, Tenure and Age, covers England and Wales, and goes down to OA level.
obviously its available through the NOMIS API and i would like us to suck that into COGS rather than download and transform. The census team have had some very good feedback from Ahmed on the data viz and sparql and i think this has generated some buzz within the census senior management team. So the idea is to load this data into COGS and have Ahmed do a data viz on it and compare that with the CMD.
We have looked at NOMIS API already and i would like us to give consideration on what and when we could get this loaded into COGS."*

**OA**: Census **O**utput **A**reas.

In [ ]:
# Components & Libraries:
import pandas as pd
import json

pd.set_option('display.max_colwidth', -1)


In [126]:
# NOMIS RESTful: Get ID for Dataset from NOMIS Data Families:

# Get Family List:
baseURL_ID = 'https://www.nomisweb.co.uk/api/v01/dataset/def.sdmx.json'
df_ID = pd.read_json(baseURL_ID)
display(df_ID)

i = 0
while True:
    str_df_ID = str(df_ID.structure[2]['keyfamily'][i]['name']['value'])
    if 'General health by tenure by age' in str_df_ID:
        print(str_df_ID + ' -- ID: ' + str(df_ID.structure[2]['keyfamily'][i]['id']))
        str_dataset_id = str(df_ID.structure[2]['keyfamily'][i]['id'])
    i += 1


,structure
common,http://www.SDMX.org/resources/SDMXML/schemas/v...
header,"{'id': 'none', 'prepared': '2020-07-30T13:52:1..."
keyfamilies,"{'keyfamily': [{'agencyid': 'NOMIS', 'annotati..."
schemalocation,http://sdmx.org/docs/2_0/SDMXMessage.xsd
structure,http://www.SDMX.org/resources/SDMXML/schemas/v...
xmlns,http://www.SDMX.org/resources/SDMXML/schemas/v...
xsi,http://www.w3.org/2001/XMLSchema-instance


LC3409EW - General health by tenure by age -- ID: NM_1404_1


IndexError: list index out of range

In [182]:
# NOMIS RESTful: Get Parent Level Geography for Dataset:

# Get Geography sets using our ID:
baseURL_Geography = 'https://www.nomisweb.co.uk/api/v01/dataset/' + str_dataset_id + '/geography.def.sdmx.json'
df_Geography = pd.read_json(baseURL_Geography)
display(df_Geography)

i = 0
while True:
    str_df_Geog_ID = str(df_Geography.structure[0]['codelist'][0]['code'][i]['description'])
    if ('England' in str_df_Geog_ID): # Dodgy code - would pick up 'England and Wales' for example!
        print(str_df_Geog_ID + ' -- ID: ' + str(df_Geography.structure[0]['codelist'][0]['code'][i]['value']))
        str_df_Geog_ID_England = str(df_Geography.structure[0]['codelist'][0]['code'][i]['value'])
    if 'Wales' in str_df_Geog_ID:
        print(str_df_Geog_ID + ' -- ID: ' + str(df_Geography.structure[0]['codelist'][0]['code'][i]['value']))
        str_df_Geog_ID_Wales = str(df_Geography.structure[0]['codelist'][0]['code'][i]['value'])
    i += 1


,structure
codelists,"{'codelist': [{'agencyid': 'NOMIS', 'id': 'CL_..."
common,http://www.SDMX.org/resources/SDMXML/schemas/v...
header,"{'id': 'NM_1404_1', 'prepared': '2020-07-30T15..."
schemalocation,http://sdmx.org/docs/2_0/SDMXMessage.xsd
structure,http://www.SDMX.org/resources/SDMXML/schemas/v...
xmlns,http://www.SDMX.org/resources/SDMXML/schemas/v...
xsi,http://www.w3.org/2001/XMLSchema-instance


{'value': 'England and Wales', 'lang': 'en'} -- ID: 2092957703
{'value': 'England and Wales', 'lang': 'en'} -- ID: 2092957703
{'value': 'England', 'lang': 'en'} -- ID: 2092957699
{'value': 'Wales', 'lang': 'en'} -- ID: 2092957700


IndexError: list index out of range

In [222]:
# NOMIS RESTful: Get Hierarchical Geography for Dataset using Parent Geography IDs:

# Get Geography sets using Parent Level Geography IDs for WALES:
baseURL_Geography_Wales = 'https://www.nomisweb.co.uk/api/v01/dataset/' + str_dataset_id + '/geography/' + str_df_Geog_ID_Wales + '.def.sdmx.json'
df_Geography_Wales = pd.read_json(baseURL_Geography_Wales)
display(df_Geography_Wales)

i = 0
while True:
    str_df_Geog_L1_ID = str(df_Geography_Wales.structure[0]['codelist'][0]['code'][i]['description'])
    if '2011 output areas within Wales' in str_df_Geog_L1_ID:
        print(str_df_Geog_L1_ID + ' -- ID: ' + str(df_Geography_Wales.structure[0]['codelist'][0]['code'][i]['value']))
        str_df_Geog_L1_ID_Wales = str(df_Geography_Wales.structure[0]['codelist'][0]['code'][i]['value'])
    i += 1


,structure
codelists,"{'codelist': [{'agencyid': 'NOMIS', 'id': 'CL_..."
common,http://www.SDMX.org/resources/SDMXML/schemas/v...
header,"{'id': 'NM_1404_1', 'prepared': '2020-07-30T16..."
schemalocation,http://sdmx.org/docs/2_0/SDMXMessage.xsd
structure,http://www.SDMX.org/resources/SDMXML/schemas/v...
xmlns,http://www.SDMX.org/resources/SDMXML/schemas/v...
xsi,http://www.w3.org/2001/XMLSchema-instance


{'value': '2011 output areas within Wales', 'lang': 'en'} -- ID: 2092957700TYPE299


IndexError: list index out of range

In [225]:
# NOMIS RESTful: Get Hierarchical Geography for Dataset using Parent Geography IDs:

# Get Geography sets using Parent Level Geography IDs for England:
baseURL_Geography_England = 'https://www.nomisweb.co.uk/api/v01/dataset/' + str_dataset_id + '/geography/' + str_df_Geog_ID_England + '.def.sdmx.json'
df_Geography_England = pd.read_json(baseURL_Geography_England)
display(df_Geography_England)

i = 0
while True:
    str_df_Geog_L1_ID = str(df_Geography_England.structure[0]['codelist'][0]['code'][i]['description'])
    if '2011 output areas within England' in str_df_Geog_L1_ID:
        print(str_df_Geog_L1_ID + ' -- ID: ' + str(df_Geography_England.structure[0]['codelist'][0]['code'][i]['value']))
        str_df_Geog_L1_ID_England = str(df_Geography_England.structure[0]['codelist'][0]['code'][i]['value'])
    i += 1


,structure
codelists,"{'codelist': [{'agencyid': 'NOMIS', 'id': 'CL_..."
common,http://www.SDMX.org/resources/SDMXML/schemas/v...
header,"{'id': 'NM_1404_1', 'prepared': '2020-07-30T16..."
schemalocation,http://sdmx.org/docs/2_0/SDMXMessage.xsd
structure,http://www.SDMX.org/resources/SDMXML/schemas/v...
xmlns,http://www.SDMX.org/resources/SDMXML/schemas/v...
xsi,http://www.w3.org/2001/XMLSchema-instance


{'value': '2011 output areas within England', 'lang': 'en'} -- ID: 2092957699TYPE299


IndexError: list index out of range

In [247]:
# NOMIS RESTful: Get Level 2 Geography Codes for Dataset using Level 1 Geography IDs:

# Get Geography sets using Level 1 Geography IDs for WALES:
baseURL_Geography_Wales_L1 = 'https://www.nomisweb.co.uk/api/v01/dataset/' + str_dataset_id + '/geography/' + str_df_Geog_L1_ID_Wales + '.def.sdmx.json'
df_Geography_Wales_L2 = pd.read_json(baseURL_Geography_Wales_L1)
display(df_Geography_Wales_L2)

i = 0
str_df_Geog_L2_ID_Wales_LowerRange = 'W99999999999999999'
str_df_Geog_L2_ID_Wales_HigherRange = ''
while True:
    str_df_Geog_L2_ID = str(df_Geography_Wales_L2.structure[0]['codelist'][0]['code'][i]['description']['value'])
    if str_df_Geog_L2_ID < str_df_Geog_L2_ID_Wales_LowerRange:
        str_df_Geog_L2_ID_Wales_LowerRange = str_df_Geog_L2_ID
    if str_df_Geog_L2_ID > str_df_Geog_L2_ID_Wales_HigherRange:
        str_df_Geog_L2_ID_Wales_HigherRange = str_df_Geog_L2_ID       
    i += 1


,structure
codelists,"{'codelist': [{'agencyid': 'NOMIS', 'id': 'CL_..."
common,http://www.SDMX.org/resources/SDMXML/schemas/v...
header,"{'id': 'NM_1404_1', 'prepared': '2020-07-30T16..."
schemalocation,http://sdmx.org/docs/2_0/SDMXMessage.xsd
structure,http://www.SDMX.org/resources/SDMXML/schemas/v...
xmlns,http://www.SDMX.org/resources/SDMXML/schemas/v...
xsi,http://www.w3.org/2001/XMLSchema-instance


IndexError: list index out of range

In [250]:
# NOMIS RESTful: Get Level 2 Geography Codes for Dataset using Level 1 Geography IDs:

# Get Geography sets using Level 1 Geography IDs for ENGLAND:
baseURL_Geography_England_L1 = 'https://www.nomisweb.co.uk/api/v01/dataset/' + str_dataset_id + '/geography/' + str_df_Geog_L1_ID_England + '.def.sdmx.json'
df_Geography_England_L2 = pd.read_json(baseURL_Geography_England_L1)
display(df_Geography_England_L2)

i = 0
str_df_Geog_L2_ID_England_LowerRange = 'E99999999999999999'
str_df_Geog_L2_ID_England_HigherRange = ''
while True:
    str_df_Geog_L2_ID = str(df_Geography_England_L2.structure[0]['codelist'][0]['code'][i]['description']['value'])
    if str_df_Geog_L2_ID < str_df_Geog_L2_ID_England_LowerRange:
        str_df_Geog_L2_ID_England_LowerRange = str_df_Geog_L2_ID
    if str_df_Geog_L2_ID > str_df_Geog_L2_ID_England_HigherRange:
        str_df_Geog_L2_ID_England_HigherRange = str_df_Geog_L2_ID       
    i += 1


,structure
codelists,"{'codelist': [{'agencyid': 'NOMIS', 'id': 'CL_..."
common,http://www.SDMX.org/resources/SDMXML/schemas/v...
header,"{'id': 'NM_1404_1', 'prepared': '2020-07-30T16..."
schemalocation,http://sdmx.org/docs/2_0/SDMXMessage.xsd
structure,http://www.SDMX.org/resources/SDMXML/schemas/v...
xmlns,http://www.SDMX.org/resources/SDMXML/schemas/v...
xsi,http://www.w3.org/2001/XMLSchema-instance


IndexError: list index out of range

In [251]:
print(str_df_Geog_L2_ID_Wales_LowerRange)
print(str_df_Geog_L2_ID_Wales_HigherRange)
print(str_df_Geog_L2_ID_England_LowerRange)
print(str_df_Geog_L2_ID_England_HigherRange)

W00000001
W00010265
E00000001
E00176774


In [254]:
# NOMIS RESTful: Get Tabular Data using Level 2 Geography Code IDs:

# Get Data for WALES:
baseURL_Geography_Wales_Data = 'https://www.nomisweb.co.uk/api/v01/dataset/' + str_dataset_id + '.data.csv?date=latest&geography=' + str_df_Geog_L2_ID_Wales_LowerRange + '...' + str_df_Geog_L2_ID_Wales_HigherRange + '&c_tenhuk11=2,3,5,6&c_age=1...4&c_health=1...3&measures=20100&select=date_name,geography_name,geography_code,c_tenhuk11_name,c_age_name,c_health_name,measures_name,obs_value,obs_status_name,record_offset,record_count'

additionalURL = ''
intRecordController = 0
final_df_wales = pd.DataFrame()

# The links below are limited to the first 25,000 cells per call.
while True:
    additionalURL = '&RecordOffset=' + str(intRecordController)
    concatenatedURL = baseURL_Geography_Wales_Data + additionalURL
    dataframe = pd.read_csv(concatenatedURL)
    if (dataframe.empty):
        break
    
    #display(dataframe)
    frames = [final_df_wales, dataframe]
    final_df_wales = pd.concat(frames)
    intRecordController = intRecordController + 25000
    
display(final_df_wales)  



,DATE_NAME,GEOGRAPHY_NAME,GEOGRAPHY_CODE,C_TENHUK11_NAME,C_AGE_NAME,C_HEALTH_NAME,MEASURES_NAME,OBS_VALUE,OBS_STATUS_NAME,RECORD_OFFSET,RECORD_COUNT
0,2011,W00000001,W00000001,Owned: Owned outright,Age 0 to 15,Very good or good health,Value,9,Normal Value,0,481728
1,2011,W00000001,W00000001,Owned: Owned outright,Age 0 to 15,Fair health,Value,0,Normal Value,1,481728
2,2011,W00000001,W00000001,Owned: Owned outright,Age 0 to 15,Bad or very bad health,Value,1,Normal Value,2,481728
3,2011,W00000001,W00000001,Owned: Owned outright,Age 16 to 49,Very good or good health,Value,21,Normal Value,3,481728
4,2011,W00000001,W00000001,Owned: Owned outright,Age 16 to 49,Fair health,Value,3,Normal Value,4,481728
5,2011,W00000001,W00000001,Owned: Owned outright,Age 16 to 49,Bad or very bad health,Value,0,Normal Value,5,481728
6,2011,W00000001,W00000001,Owned: Owned outright,Age 50 to 64,Very good or good health,Value,30,Normal Value,6,481728
7,2011,W00000001,W00000001,Owned: Owned outright,Age 50 to 64,Fair health,Value,4,Normal Value,7,481728
8,2011,W00000001,W00000001,Owned: Owned outright,Age 50 to 64,Bad or very bad health,Value,1,Normal Value,8,481728
9,2011,W00000001,W00000001,Owned: Owned outright,Age 65 and over,Very good or good health,Value,28,Normal Value,9,481728


In [270]:
# NOMIS RESTful: Get Tabular Data using Level 2 Geography Code IDs:

# Get Data for ENGLAND:
baseURL_Geography_England_Data = 'https://www.nomisweb.co.uk/api/v01/dataset/' + str_dataset_id + '.data.csv?date=latest&geography=' + str_df_Geog_L2_ID_England_LowerRange + '...' + str_df_Geog_L2_ID_England_HigherRange + '&c_tenhuk11=2,3,5,6&c_age=1...4&c_health=1...3&measures=20100&select=date_name,geography_name,geography_code,c_tenhuk11_name,c_age_name,c_health_name,measures_name,obs_value,obs_status_name,record_offset,record_count'

additionalURL = ''
intRecordController = 0
final_df_england = pd.DataFrame()

# The links below are limited to the first 25,000 cells per call.
while True:
    additionalURL = '&RecordOffset=' + str(intRecordController)
    concatenatedURL = baseURL_Geography_England_Data + additionalURL
    dataframe = pd.read_csv(concatenatedURL)
    if (dataframe.empty):
        break
    
    #display(dataframe)
    # Additional code for England due to large datasets:
    # For a very basic progress monitor (no fancy code this time round):
    print("Just received record:" + (dataframe.tail(1).RECORD_OFFSET.to_string(index=False)) + ' of ' + (dataframe.tail(1).RECORD_COUNT.to_string(index=False)) + ' >>> ' + str(round((int(dataframe.tail(1).RECORD_OFFSET.to_string(index=False)) / int(dataframe.tail(1).RECORD_COUNT.to_string(index=False)) * 100),2)) + '%...')
    frames = [final_df_england, dataframe]
    final_df_england = pd.concat(frames)
    intRecordController = intRecordController + 25000
    
display(final_df_england)


Just received record: 24999 of  8225856 >>> 0.3%...
Just received record: 49999 of  8225856 >>> 0.61%...
Just received record: 74999 of  8225856 >>> 0.91%...
Just received record: 99999 of  8225856 >>> 1.22%...
Just received record: 124999 of  8225856 >>> 1.52%...
Just received record: 149999 of  8225856 >>> 1.82%...
Just received record: 174999 of  8225856 >>> 2.13%...
Just received record: 199999 of  8225856 >>> 2.43%...
Just received record: 224999 of  8225856 >>> 2.74%...
Just received record: 249999 of  8225856 >>> 3.04%...
Just received record: 274999 of  8225856 >>> 3.34%...
Just received record: 299999 of  8225856 >>> 3.65%...
Just received record: 324999 of  8225856 >>> 3.95%...
Just received record: 349999 of  8225856 >>> 4.25%...
Just received record: 374999 of  8225856 >>> 4.56%...
Just received record: 399999 of  8225856 >>> 4.86%...
Just received record: 424999 of  8225856 >>> 5.17%...
Just received record: 449999 of  8225856 >>> 5.47%...
Just received record: 474999 of  

Just received record: 3724999 of  8225856 >>> 45.28%...
Just received record: 3749999 of  8225856 >>> 45.59%...
Just received record: 3774999 of  8225856 >>> 45.89%...
Just received record: 3799999 of  8225856 >>> 46.2%...
Just received record: 3824999 of  8225856 >>> 46.5%...
Just received record: 3849999 of  8225856 >>> 46.8%...
Just received record: 3874999 of  8225856 >>> 47.11%...
Just received record: 3899999 of  8225856 >>> 47.41%...
Just received record: 3924999 of  8225856 >>> 47.72%...
Just received record: 3949999 of  8225856 >>> 48.02%...
Just received record: 3974999 of  8225856 >>> 48.32%...
Just received record: 3999999 of  8225856 >>> 48.63%...
Just received record: 4024999 of  8225856 >>> 48.93%...
Just received record: 4049999 of  8225856 >>> 49.23%...
Just received record: 4074999 of  8225856 >>> 49.54%...
Just received record: 4099999 of  8225856 >>> 49.84%...
Just received record: 4124999 of  8225856 >>> 50.15%...
Just received record: 4149999 of  8225856 >>> 50.45

Just received record: 7399999 of  8225856 >>> 89.96%...
Just received record: 7424999 of  8225856 >>> 90.26%...
Just received record: 7449999 of  8225856 >>> 90.57%...
Just received record: 7474999 of  8225856 >>> 90.87%...
Just received record: 7499999 of  8225856 >>> 91.18%...
Just received record: 7524999 of  8225856 >>> 91.48%...
Just received record: 7549999 of  8225856 >>> 91.78%...
Just received record: 7574999 of  8225856 >>> 92.09%...
Just received record: 7599999 of  8225856 >>> 92.39%...
Just received record: 7624999 of  8225856 >>> 92.7%...
Just received record: 7649999 of  8225856 >>> 93.0%...
Just received record: 7674999 of  8225856 >>> 93.3%...
Just received record: 7699999 of  8225856 >>> 93.61%...
Just received record: 7724999 of  8225856 >>> 93.91%...
Just received record: 7749999 of  8225856 >>> 94.22%...
Just received record: 7774999 of  8225856 >>> 94.52%...
Just received record: 7799999 of  8225856 >>> 94.82%...
Just received record: 7824999 of  8225856 >>> 95.13

,DATE_NAME,GEOGRAPHY_NAME,GEOGRAPHY_CODE,C_TENHUK11_NAME,C_AGE_NAME,C_HEALTH_NAME,MEASURES_NAME,OBS_VALUE,OBS_STATUS_NAME,RECORD_OFFSET,RECORD_COUNT
0,2011,E00000001,E00000001,Owned: Owned outright,Age 0 to 15,Very good or good health,Value,4,Normal Value,0,8225856
1,2011,E00000001,E00000001,Owned: Owned outright,Age 0 to 15,Fair health,Value,0,Normal Value,1,8225856
2,2011,E00000001,E00000001,Owned: Owned outright,Age 0 to 15,Bad or very bad health,Value,0,Normal Value,2,8225856
3,2011,E00000001,E00000001,Owned: Owned outright,Age 16 to 49,Very good or good health,Value,19,Normal Value,3,8225856
4,2011,E00000001,E00000001,Owned: Owned outright,Age 16 to 49,Fair health,Value,1,Normal Value,4,8225856
5,2011,E00000001,E00000001,Owned: Owned outright,Age 16 to 49,Bad or very bad health,Value,0,Normal Value,5,8225856
6,2011,E00000001,E00000001,Owned: Owned outright,Age 50 to 64,Very good or good health,Value,28,Normal Value,6,8225856
7,2011,E00000001,E00000001,Owned: Owned outright,Age 50 to 64,Fair health,Value,1,Normal Value,7,8225856
8,2011,E00000001,E00000001,Owned: Owned outright,Age 50 to 64,Bad or very bad health,Value,1,Normal Value,8,8225856
9,2011,E00000001,E00000001,Owned: Owned outright,Age 65 and over,Very good or good health,Value,17,Normal Value,9,8225856


In [271]:
print('boo')

boo


In [277]:
final_df_wales.to_csv('final_df_wales.csv', date_format='%Y%m%d')

In [276]:
final_df_england.to_csv('final_df_england.csv', date_format='%Y%m%d')